# Machine Learning with scikit-learn

* What is Machine Learning?
  * Types of Machine Learning
* Train-Test split
* Use `sklearn` to build linear regression model
* One-Hot Encoding
* Pipelines
* Evaluation Metrics

We will be referencing the [scikit-learn docs](https://scikit-learn.org/stable/user_guide.html) and [pandas docs](https://pandas.pydata.org/pandas-docs/stable/index.html) where relevant, and will be analyzing data from the New York Times COVID-19 US States dataset from https://github.com/nytimes/covid-19-data

**Disclaimer: Linear regression is not the most suitable algorithm for this dataset, but we are using it to illustrate how to use scikit-learn**

## What is Machine Learning?

* Learning patterns in your data without being explicitly programmed
* A function that maps features to an output

![](https://brookewenig.com/img/DL/al_ml_dl.png)

-sandbox
## Types of Machine Learning
* Supervised Learning
  * Regression <img src="https://miro.medium.com/max/640/1*LEmBCYAttxS6uI6rEyPLMQ.png" style="height: 250px; padding: 10px"/>
  * Classification
    <img src="https://cdn3-www.dogtime.com/assets/uploads/2018/10/puppies-cover.jpg" style="height: 250px; padding: 10px"/>
    <img src="https://images.unsplash.com/photo-1529778873920-4da4926a72c2?ixlib=rb-1.2.1&w=1000&q=80" style="height: 250px; padding: 10px"/>
* Unsupervised Learning
<img src="https://www.iotforall.com/wp-content/uploads/2018/01/Screen-Shot-2018-01-17-at-8.10.14-PM.png" style="height: 250px; padding: 10px"/>
* Reinforcement Learning
<img src="https://brookewenig.com/img/ReinforcementLearning/Rl_agent.png" style="height: 250px; padding: 10px"/>

Today we're going to start simple and focus on a supervised learning (regression) problem. Here we will use a linear regression model to predict the number of deaths resulting from COVID-19.

In [5]:
%fs ls databricks-datasets/COVID/covid-19-data/us-states.csv

path,name,size
dbfs:/databricks-datasets/COVID/covid-19-data/us-states.csv,us-states.csv,71682


In [6]:
import pandas as pd

df = pd.read_csv("/dbfs/databricks-datasets/COVID/covid-19-data/us-states.csv")
df.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [7]:
df.shape

Out[50]: (2385, 5)

## Relationship between Cases & Deaths

In [9]:
# Filter to 2020-04-14
df_04_14 = df[df["date"] == "2020-04-14"]

ax = df_04_14.plot(x="cases", y="deaths", kind="scatter", 
                   figsize=(12,8), s=100, title="Deaths vs Cases on 2020-04-14 - All States")

df_04_14[["cases", "deaths", "state"]].apply(lambda row: ax.text(*row), axis=1);

## New York & New Jersey are Outliers

In [11]:
# Filter to states that are NOT New York and NOT New Jersey
not_ny = df[(df["state"] != "New York") & (df["state"] != "New Jersey")]
not_ny.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [12]:
# Filter to 2020-04-14
not_ny_04_14 = not_ny[not_ny["date"] == "2020-04-14"]

ax = not_ny_04_14.plot(x="cases", y="deaths", kind="scatter", 
                   figsize=(12,8), s=50, title="Deaths vs Cases on 2020-04-14 - All States but NY and NJ")

not_ny_04_14[["cases", "deaths", "state"]].apply(lambda row: ax.text(*row), axis=1);

## New York verus California COVID-19 deaths comparison

In [14]:
df_ny_cali = df[(df["state"] == "New York") | (df["state"] == "California")]

# Let's pivot our df_ny_cali DataFrame so that we can plot deaths over time for both states
df_ny_cali_pivot = df_ny_cali.pivot(index='date', columns='state', values='deaths').fillna(0)
df_ny_cali_pivot

state,California,New York
date,,
2020-01-25,0.0,0.0
2020-01-26,0.0,0.0
2020-01-27,0.0,0.0
2020-01-28,0.0,0.0
2020-01-29,0.0,0.0
2020-01-30,0.0,0.0
2020-01-31,0.0,0.0
2020-02-01,0.0,0.0
2020-02-02,0.0,0.0


In [15]:
df_ny_cali_pivot.plot.line(title="Deaths 2020-01-25 to 2020-01-25 - CA and NY", figsize=(12,8))

## Train-Test Split

![](https://brookewenig.com/img/IntroML/trainTest.png)

Because this is temporal data, instead of doing a random split, we will use data from March 1 to April 7 to train our model, and test our model by predicting values for April 8 - 14.

In [18]:
train_df = df[(df["date"] >= "2020-03-01") & (df["date"] <= "2020-04-07")]
test_df = df[df["date"] > "2020-04-07"]

X_train = train_df[["cases"]]
y_train = train_df["deaths"]

X_test = test_df[["cases"]]
y_test = test_df["deaths"]

## Linear Regression

* Goal: Find the line of best fit
$$\hat{y} = w_0 + w_1x$$

$$\{y} ≈ \hat{y} + ϵ$$
* *x*: feature
* *y*: label

![](https://miro.medium.com/max/640/1*LEmBCYAttxS6uI6rEyPLMQ.png)

Here we will be fitting a [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) model from scikit-learn.

In [21]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression().fit(X_train, y_train)
print(f"num_deaths = {lr.intercept_:.4f} + {lr.coef_[0]:.4f}*cases")

num_deaths = -8.9911 + 0.0293*cases

Hmmm... if we have no cases, then there should be no deaths caused by COVID-19, so let's set the intercept to be 0.

In [23]:
lr = LinearRegression(fit_intercept=False).fit(X_train, y_train)
print(f"num_deaths = {lr.coef_[0]:.4f}*cases")

num_deaths = 0.0290*cases

So this model is implying that there is a 2.9% mortality rate in our dataset. But we know that some states have higher mortality rates than others. Let's include the state as a feature!

## One-Hot Encoding
How do we handle non-numeric features, such as the state?

One idea:
* Create single numerical feature to represent non-numeric one
* Categorical features:
  * state = {'New York', 'California', 'Louisiana'}
  * 'New York' = 1, 'California' = 2, 'Louisiana' = 3
  
BUT this implies California is 2x New York!

Better idea:
* Create a ‘dummy’ feature for each category
* 'New York' => [1, 0, 0], 'California' => [0, 1, 0], 'Louisiana' => [0, 0, 1]

This technique is known as ["One Hot Encoding"](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)

In [26]:
from sklearn.preprocessing import OneHotEncoder

X_train = train_df[["cases", "state"]]
X_test = test_df[["cases", "state"]]

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
enc.fit(X_train).transform(X_train)

Out[61]: array([[1., 0., 0., ..., 0., 0., 0.],
 [0., 0., 0., ..., 0., 0., 0.],
 [0., 1., 0., ..., 0., 0., 0.],
 ...,
 [0., 0., 0., ..., 1., 0., 0.],
 [0., 0., 0., ..., 0., 1., 0.],
 [0., 0., 0., ..., 0., 0., 1.]])

Let's check the shape

In [28]:
enc.fit(X_train).transform(X_train).shape

Out[62]: (1754, 914)

Yikes! It one-hot encoded the cases variable too

In [30]:
enc.categories_

Out[63]: [array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 89,
 90, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105,
 106, 107, 108, 109, 110, 112, 113, 114, 115, 116, 118, 119, 120,
 121, 122, 123, 124, 126, 127, 129, 130, 131, 132, 133, 136, 137,
 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 150, 151, 152,
 153, 154, 155, 156, 157, 158, 159, 161, 162, 164, 165, 166, 168,
 169, 171, 172, 173, 174, 175, 176, 177, 179, 182, 183, 184, 185,
 186, 187, 189, 190, 191, 193, 194, 195, 196, 197, 198, 200, 201,
 202, 203, 204, 207, 208, 209, 210, 211, 212, 214, 216, 217, 218,
 219, 221, 223, 224, 225, 231, 232, 234, 235, 237, 239, 240, 241,
 242, 245, 247, 248, 249, 252, 253, 254, 255, 256, 257, 258, 259,
 261, 262, 264, 265, 266, 267, 268, 269, 271, 273, 274, 275, 278,
 279, 280, 281, 282, 283, 286, 287, 288, 290, 293, 294, 296, 298,
 299, 302, 303, 304, 308, 310, 312, 314, 315, 316, 317, 319, 320,
 321, 322, 324, 326, 327, 328, 332, 334, 336, 338, 339, 342, 344,
 345, 346, 349, 351, 363, 364, 366, 367, 368, 369, 373, 376, 377,
 378, 381, 382, 385, 386, 388, 389, 391, 392, 393, 394, 398, 399,
 402, 403, 405, 408, 409, 412, 413, 414, 415, 418, 419, 420, 421,
 423, 424, 427, 429, 432, 434, 438, 439, 441, 444, 449, 450, 452,
 456, 461, 470, 475, 478, 479, 480, 481, 482, 487, 488, 490, 495,
 498, 499, 504, 507, 508, 511, 512, 513, 519, 522, 525, 526, 535,
 537, 538, 539, 540, 543, 547, 548, 549, 552, 554, 563, 564, 566,
 573, 575, 576, 579, 581, 583, 586, 588, 590, 591, 593, 604, 606,
 608, 609, 610, 614, 618, 620, 621, 623, 624, 629, 632, 638, 639,
 644, 646, 653, 656, 657, 660, 665, 669, 671, 673, 675, 679, 682,
 683, 686, 689, 690, 699, 701, 706, 711, 715, 719, 720, 721, 728,
 732, 735, 736, 738, 740, 742, 743, 747, 751, 754, 755, 757, 759,
 763, 764, 770, 773, 774, 775, 777, 783, 786, 787, 789, 794, 800,
 806, 826, 830, 837, 838, 848, 851, 853, 857, 865, 867, 868, 875,
 879, 887, 890, 891, 893, 896, 899, 902, 903, 905, 908, 912, 917,
 920, 922, 925, 927, 928, 929, 930, 935, 937, 946, 947, 950, 972,
 981, 982, 986, 988, 997, 998, 999, 1000, 1012, 1013, 1021, 1026,
 1033, 1042, 1044, 1047, 1048, 1050, 1057, 1066, 1067, 1068, 1069,
 1073, 1074, 1077, 1082, 1083, 1086, 1097, 1101, 1106, 1113, 1119,
 1120, 1132, 1144, 1151, 1159, 1167, 1169, 1170, 1172, 1177, 1180,
 1181, 1210, 1211, 1222, 1228, 1229, 1232, 1244, 1250, 1252, 1260,
 1267, 1270, 1279, 1283, 1285, 1289, 1291, 1293, 1298, 1304, 1324,
 1327, 1336, 1351, 1358, 1364, 1374, 1387, 1388, 1404, 1406, 1413,
 1414, 1428, 1432, 1455, 1458, 1467, 1472, 1511, 1514, 1524, 1528,
 1535, 1543, 1544, 1549, 1550, 1554, 1570, 1581, 1582, 1600, 1605,
 1633, 1638, 1643, 1655, 1662, 1665, 1675, 1690, 1700, 1706, 1721,
 1730, 1735, 1738, 1752, 1758, 1769, 1788, 1791, 1795, 1834, 1838,
 1841, 1844, 1851, 1857, 1875, 1880, 1914, 1915, 1916, 1917, 1933,
 1942, 1965, 1971, 1985, 1993, 2006, 2012, 2019, 2049, 2061, 2093,
 2101, 2112, 2113, 2159, 2161, 2197, 2198, 2199, 2218, 2232, 2240,
 2267, 2269, 2291, 2294, 2305, 2315, 2331, 2367, 2382, 2402, 2407,
 2417, 2440, 2447, 2465, 2469, 2477, 2538, 2547, 2565, 2571, 2575,
 2578, 2585, 2628, 2637, 2644, 2683, 2690, 2712, 2722, 2746, 2758,
 2815, 2844, 2869, 2870, 2878, 2902, 2990, 3029, 3032, 3037, 3039,
 3043, 3125, 3128, 3183, 3198, 3208, 3220, 3240, 3265, 3312, 3315,
 3333, 3346, 3437, 3441, 3538, 3540, 3547, 3557, 3576, 3609, 3657,
 3675, 3728, 3739, 3753, 3770, 3824, 3953, 4010, 4025, 4038, 4043,
 4045, 4060, 4117, 4152, 4156, 4182, 4257, 4261, 4311, 4371, 4402,
 4411, 4450, 4574, 4613, 4635, 4748, 4782, 4880, 4896, 4914, 4915,
 4942, 4950, 4955, 4956, 4997, 5070, 5172, 5179, 5237, 5276, 5292,
 5429, 5444, 5486, 5507, 5565, 5588, 5669, 5675, 569

We need the [column transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) to only apply the one hot encoding to a single column.

In [32]:
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer([("enc", enc, ["state"])], remainder="passthrough")
ct.fit_transform(X_train).shape

Out[64]: (1754, 56)

## Pipelines

We can chain together a series of data transformations with a [pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html). This way we also ensure that whatever operations we apply to our training set, we also apply in the same order to our test set.

In [34]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline(steps=[("ct", ct), ("lr", lr)])
pipeline_model = pipeline.fit(X_train, y_train)

y_pred = pipeline_model.predict(X_test)

## How are the different states performing?

You'll notice that by adding in additional features, our coefficient for our cases feature changed as well.

In [36]:
print(f"num_deaths = {pipeline_model.steps[1][1].coef_[-1]:.4f}*cases + state_coef")

num_deaths = 0.0311*cases + state_coef

In [37]:
import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format)

categories = pipeline_model.steps[0][1].transformers[0][1].categories_[1]

pd.DataFrame(zip(categories, pipeline_model.steps[1][1].coef_[:-1]), columns=["State", "Coefficient"])

,State,Coefficient
0,Alabama,-7.92
1,Alaska,-1.32
2,Arizona,-4.40
3,Arkansas,-5.80
4,California,-35.93
5,Colorado,-9.69
6,Connecticut,-5.97
7,Delaware,-2.21
8,District of Columbia,-3.91
9,Florida,-45.02


## Evaluation Metrics

![](https://brookewenig.com/img/IntroML/RMSE.png)

Let's compute the MSE and RMSE for our dataset using the [sklearn.metrics](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html?highlight=mean_squared_error).

In [40]:
from sklearn.metrics import mean_squared_error
import numpy as np

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"MSE is {mse:.1f}, RMSE is {rmse:.1f}")

MSE is 217339.9, RMSE is 466.2

## Visualize Predictions

In [42]:
pred = pd.concat([test_df.reset_index(drop=True), pd.DataFrame(y_pred, columns=["predicted_deaths"])], axis=1)
pred

,date,state,fips,cases,deaths,predicted_deaths
0,2020-04-08,Alabama,1,2499,67,69.80
1,2020-04-08,Alaska,2,224,5,5.64
2,2020-04-08,Arizona,4,2726,80,80.38
3,2020-04-08,Arkansas,5,1077,18,27.69
4,2020-04-08,California,6,19043,506,556.32
5,2020-04-08,Colorado,8,5655,192,166.18
6,2020-04-08,Connecticut,9,8781,335,267.13
7,2020-04-08,Delaware,10,1116,19,32.50
8,2020-04-08,District of Columbia,11,1440,27,40.87
9,2020-04-08,Florida,12,15690,322,442.94


Voila! You have successfully built a machine learning pipeline using scikit-learn!

To keep exploring with scikit-learn, checkout the datasets at [UCI ML Repository](https://archive.ics.uci.edu/ml/index.php) and [Kaggle](https://www.kaggle.com/)!